In [72]:
# Célula 1: Importações
import pandas as pd
import csv, os, zipfile
import pickle
import shutil
import sys
import requests
import tempfile
from datetime import date, timedelta

In [ ]:
# Célula 2: Definições Globais
instituicao = 'Instituto Federal de São Paulo'

anos_para_baixar = ['2022', '2023'] # Anos para baixar os dados de viagem - CONFIGURAR AQUI!

funcoes_validas = ["CD-0001", "CD-0002", "CD-0003", "CD-0004"] # Cargo de Direção - Funções válidas para filtro

pasta_temp_viagens = "temp_viagens_download" # Pasta temporária para downloads de viagens
pasta_temp_raiz_downloads = "temp_downloads_raiz" # Pasta temporária raiz para downloads SIAPE

os.makedirs(pasta_temp_viagens, exist_ok=True) # Criar pasta se não existir
os.makedirs(pasta_temp_raiz_downloads, exist_ok=True) # Criar pasta se não existir

last_successful_df_serv_global = None # Variável global para armazenar o último df_serv válido
print('Definições carregadas com êxito')

Definições carregadas com êxito


In [74]:
# Célula 3: Funções para Encontrar Data Valida e Download de ZIP de Viagens (REFORMULADA)

def encontrar_data_valida_viagens(ano, pasta_destino=pasta_temp_viagens, tentativas=8):
    """
    Tenta encontrar uma data válida para download do arquivo ZIP de viagens para o ano especificado,
    testando URLs com retentativas de data. Retorna a data válida (YYYYMMDD) ou None.
    """
    data_base = date.today()
    print(f"DEBUG: Iniciando busca por data válida para o ano: {ano}. Data base inicial: {data_base}") # DEBUG

    for i in range(tentativas):
        data_referencia = data_base - timedelta(days=i)
        data_str = data_referencia.strftime('%Y%m%d')
        url_zip_viagens = f'https://dadosabertos-download.cgu.gov.br/PortalDaTransparencia/saida/viagens/{ano}_{data_str}_Viagens.zip'

        print(f"DEBUG: Testando data {i+1}/{tentativas}: {data_str} - URL: {url_zip_viagens}") # DEBUG
        try:
            response = requests.head(url_zip_viagens, timeout=10) # Usando HEAD request para verificar mais rápido
            response.raise_for_status() # Raise HTTPError para status codes ruins (4xx ou 5xx)

            if response.status_code == 200:
                print(f"DEBUG: Data válida encontrada: {data_str} para o ano {ano}") # DEBUG
                print(f"Data válida para download encontrada: {data_str}")
                return data_str # Retorna a data válida (YYYYMMDD)
            else:
                print(f"DEBUG: Status code {response.status_code} para {url_zip_viagens}") # DEBUG
                continue # Próxima tentativa com data anterior

        except requests.exceptions.HTTPError as http_err:
            if http_err.response.status_code == 403:
                print(f"DEBUG: Acesso proibido (403) para {url_zip_viagens} na data {data_str}. Tentando data anterior.") # DEBUG
            else:
                print(f"DEBUG: Erro HTTP {http_err.response.status_code} ao testar {url_zip_viagens}: {http_err}") # DEBUG
            continue # Próxima tentativa com data anterior
        except requests.exceptions.RequestException as e:
            print(f"DEBUG: Erro ao testar URL {url_zip_viagens}: {e}") # DEBUG
            continue # Próxima tentativa com data anterior

    print(f"DEBUG: Nenhuma data válida encontrada para o ano {ano} após {tentativas} tentativas.") # DEBUG
    print(f"Falha ao encontrar data válida para download do arquivo de viagens para {ano} após {tentativas} tentativas.")
    return None # Nenhuma data válida encontrada


def download_arquivo_viagens_com_data(ano, data_valida, pasta_destino=pasta_temp_viagens):
    """
    Baixa o arquivo ZIP de viagens para o ano especificado, utilizando a data válida já identificada.
    """
    data_str = data_valida
    url_zip_viagens = f'https://dadosabertos-download.cgu.gov.br/PortalDaTransparencia/saida/viagens/{ano}_{data_str}_Viagens.zip'
    nome_arquivo_zip = os.path.join(pasta_destino, f'{ano}_Viagens_{data_str}.zip')

    print(f"Iniciando download do arquivo de viagens para {ano} com data válida {data_str}: {url_zip_viagens}")

    try:
        response = requests.get(url_zip_viagens, stream=True)
        response.raise_for_status()

        with open(nome_arquivo_zip, 'wb') as arquivo_zip:
            for chunk in response.iter_content(chunk_size=8192):
                arquivo_zip.write(chunk)
        print(f"Download bem-sucedido para: {nome_arquivo_zip}")
        return nome_arquivo_zip

    except requests.exceptions.HTTPError as http_err:
        print(f"Erro HTTP ao baixar {url_zip_viagens}: {http_err}")
    except requests.exceptions.RequestException as e:
        print(f"Erro ao baixar {url_zip_viagens}: {e}")

    print(f"Falha ao baixar arquivo de viagens para {ano} com data {data_str}.")
    return None

In [75]:
# Célula 4: Função para Carregar DataFrame de Viagens a partir de ZIP
def carregar_dataframe_viagens_csv_zip(zip_file_path, ano):
    """
    Extrai o arquivo CSV de viagens do ZIP e carrega em um DataFrame, aplicando filtros.
    """
    nome_csv_viagem = f'{ano}_Viagem.csv'
    try:
        with zipfile.ZipFile(zip_file_path, 'r') as zf:
            with zf.open(nome_csv_viagem) as csvfile:
                df = pd.read_csv(csvfile, sep=';', quotechar='"', encoding='iso-8859-1', decimal=',', thousands='.')

                df_filtrado = df[
                    (df['Situação'] == 'Realizada') &
                    (df['Função'].isin(funcoes_validas))
                ]
                print(f"DataFrame de viagens para {ano} carregado e filtrado com {len(df_filtrado)} linhas.")
                return df_filtrado
    except FileNotFoundError:
        print(f"Arquivo CSV '{nome_csv_viagem}' não encontrado dentro de {zip_file_path}")
        return None
    except Exception as e:
        print(f"Erro ao processar arquivo ZIP de viagens {zip_file_path}: {e}")
        return None
print('Função carregada com êxito')

Função carregada com êxito


In [76]:
# Célula 5: Função para Extrair CSV do ZIP (SIAPE)
def extrair_csv_do_zip(zip_file_path, pasta_temp="temp_siape_arquivos"):
    """Extrai o arquivo '_Cadastro.csv' de um arquivo ZIP."""
    try:
        os.makedirs(pasta_temp, exist_ok=True)
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            nome_csv_cadastro = next((filename for filename in zip_ref.namelist() if filename.endswith('_Cadastro.csv')), None)
            if nome_csv_cadastro:
                csv_path = os.path.join(pasta_temp, nome_csv_cadastro)
                zip_ref.extract(nome_csv_cadastro, pasta_temp)
                return csv_path
            else:
                print(f"Arquivo '_Cadastro.csv' não encontrado em {zip_file_path}")
                return None
    except Exception as e:
        print(f"Erro ao extrair CSV de {zip_file_path}: {e}")
        return None
print('Função carregada com êxito')

Função carregada com êxito


In [77]:
# Célula 6: Função para Carregar DataFrame SIAPE
def carregar_dataframe_siape(csv_path, zip_file_path):
    """Carrega o DataFrame SIAPE, aplica filtros, verifica duplicatas e limpeza inicial."""
    df_serv = None
    try:
        print(f"  Tentando ler CSV (com filtros): {csv_path}")

        chunks = pd.read_csv(
            csv_path,
            sep=';',
            quotechar='"',
            encoding='iso-8859-1',
            decimal=',',
            thousands='.',
            chunksize=1000,
            iterator=True
        )

        list_of_filtered_chunks = []
        for chunk in chunks:
            filtered_chunk = chunk[
                (chunk['ATIVIDADE'].notna()) &
                (~chunk['ATIVIDADE'].isin(["Sigiloso", "Sem informaç"])) &
                (chunk['ORG_EXERCICIO'] == instituicao)
            ]
            list_of_filtered_chunks.append(filtered_chunk)

        df_serv = pd.concat(list_of_filtered_chunks, ignore_index=True)
        print(f"  DataFrame SIAPE filtrado carregado com {len(df_serv)} linhas.")

        nome_arquivo_zip = os.path.basename(zip_file_path)
        ano_mes_siape = nome_arquivo_zip[:6]
        df_serv['siape_ano'] = ano_mes_siape[:4]
        df_serv['siape_mes'] = ano_mes_siape[4:]

        try:
            df_serv['CPF'] = df_serv['CPF'].str.strip()
        except Exception as e_cpf:
            print(f"  Erro ao limpar coluna 'CPF': {e_cpf}")
            return None

        try:
            df_serv['NOME'] = df_serv['NOME'].str.strip().str.lower()
        except Exception as e_nome:
            print(f"  Erro ao limpar coluna 'NOME': {e_nome}")
            return None

        try:
            duplicatas_serv = df_serv.duplicated(subset=['NOME', 'siape_ano', 'siape_mes'], keep=False)
            if duplicatas_serv.any():
                print(f"  AVISO: Encontradas duplicatas por NOME, siape_ano e siape_mes no arquivo SIAPE {csv_path}")
                print(df_serv[duplicatas_serv].sort_values(by=['NOME', 'siape_ano', 'siape_mes']).head())
            else:
                print(f"  Não foram encontradas duplicatas por NOME, siape_ano e siape_mes no arquivo SIAPE {csv_path}")
        except Exception as e_dup:
            print(f"  Erro ao verificar duplicatas: {e_dup}")
            return None

        return df_serv

    except Exception as e_overall:
        print(f"Erro ao carregar DataFrame SIAPE de {csv_path}: {e_overall}")
        return None
print('Função carregada com êxito')

Função carregada com êxito


In [78]:
# Célula 7: Função para Preparar DataFrame de Viagens
def preparar_dataframe_viagens(df_viagens):
    """Prepara o DataFrame de viagens, convertendo datas e limpando nomes."""
    df_viagens_preparado = df_viagens.copy()
    df_viagens_preparado['Período - Data de início'] = pd.to_datetime(df_viagens_preparado['Período - Data de início'], format='%d/%m/%Y', errors='coerce')
    df_viagens_preparado['viagem_ano_mes'] = df_viagens_preparado['Período - Data de início'].dt.strftime('%Y%m')
    df_viagens_preparado['viagem_ano'] = df_viagens_preparado['Período - Data de início'].dt.year.astype(str)
    df_viagens_preparado['viagem_mes'] = df_viagens_preparado['Período - Data de início'].dt.month.astype(str).str.zfill(2)
    df_viagens_preparado['Nome'] = df_viagens_preparado['Nome'].str.strip().str.lower()
    return df_viagens_preparado
print('Função carregada com êxito')

Função carregada com êxito


In [79]:
# Célula 8: Função para Merge de DataFrames (Viagens e SIAPE)
def merge_dataframes(df_viagens_preparado, df_serv_preparado):
    """Realiza o merge dos DataFrames de viagens e SIAPE usando 'NOME', 'ano' e 'mês'."""
    if df_viagens_preparado is None or df_serv_preparado is None:
        return None

    colunas_viagens_para_merge = [
        'Identificador do processo de viagem', 'Número da Proposta (PCDP)', 'Situação', 'Viagem Urgente',
        'Justificativa Urgência Viagem', 'Código do órgão superior', 'Nome do órgão superior',
        'Código órgão solicitante', 'Nome órgão solicitante', 'CPF viajante', 'Nome', 'Cargo', 'Função',
        'Descrição Função', 'Período - Data de início', 'Período - Data de fim', 'Destinos', 'Motivo',
        'Valor diárias', 'Valor passagens', 'Valor devolução', 'Valor outros gastos',
        'viagem_ano', 'viagem_mes'
    ]
    df_viagens_para_merge = df_viagens_preparado[colunas_viagens_para_merge].copy()

    df_merged = pd.merge(df_viagens_para_merge,
                         df_serv_preparado[['NOME', 'siape_ano', 'siape_mes', 'FUNCAO', 'ATIVIDADE', 'UORG_EXERCICIO', 'ORG_EXERCICIO']],
                         left_on=['Nome', 'viagem_ano', 'viagem_mes'],
                         right_on=['NOME', 'siape_ano', 'siape_mes'],
                         how='left',
                         suffixes=(False, False)
                         )
    return df_merged
print('Função carregada com êxito')

Função carregada com êxito


In [80]:
# Célula 9: Função para Limpar Arquivos Temporários
def limpar_arquivos_temporarios(pasta_temp="temp_siape_arquivos"):
    """Limpa a pasta temporária e seu conteúdo."""
    try:
        if os.path.exists(pasta_temp):
            shutil.rmtree(pasta_temp)
            print(f"Pasta temporária '{pasta_temp}' removida com sucesso.")
        else:
            print(f"Pasta temporária '{pasta_temp}' não encontrada, pulando remoção.")
    except FileNotFoundError:
        print(f"Pasta temporária '{pasta_temp}' não encontrada (FileNotFoundError), pulando remoção.")
    except Exception as e:
        print(f"Erro ao remover pasta temporária '{pasta_temp}': {e}")
print('Função carregada com êxito')

Função carregada com êxito


In [81]:
# Célula 10: Função para Consolidar SIAPE Anual
def consolidar_siape_anual(lista_arquivos_zip_ano):
    """Consolida arquivos SIAPE de um ano em um único DataFrame."""
    df_serv_ano_consolidado = pd.DataFrame()
    for zip_file_path in lista_arquivos_zip_ano:
        csv_path = extrair_csv_do_zip(zip_file_path)
        if csv_path:
            df_serv_mensal = carregar_dataframe_siape(csv_path, zip_file_path)
            if df_serv_mensal is not None:
                df_serv_ano_consolidado = pd.concat([df_serv_ano_consolidado, df_serv_mensal], ignore_index=True)
                try:
                    os.remove(csv_path)
                except Exception as e_remove_csv:
                    print(f"  Erro ao remover arquivo CSV {csv_path}: {e_remove_csv}")
            else:
                print(f"  Erro ao carregar dados SIAPE do arquivo: {zip_file_path}, pulando.")
        else:
            print(f"  Arquivo ZIP não encontrado ou erro na extração: {zip_file_path}, pulando.")
    return df_serv_ano_consolidado
print('Função carregada com êxito')

Função carregada com êxito


In [82]:
# Célula 11: Função para Processar Ano (SIAPE e Merge) - Refatorada e Modificada para retornar períodos faltantes
def processar_ano_refatorado(ano, df_viagens, pasta_temp_raiz="temp_siape_downloads"):
    """Processa os arquivos SIAPE de um ano, baixando-os sob demanda, e faz o merge com o dataframe de viagens.
       Retorna uma lista de períodos (ano-mês) para os quais o download do SIAPE falhou.
    """
    global last_successful_df_serv_global

    print(f"\n--- Processando ano SIAPE: {ano} ---")

    with tempfile.TemporaryDirectory(dir=pasta_temp_raiz) as temp_dir_ano:
        print(f"Pasta temporária para o ano SIAPE {ano}: {temp_dir_ano}")
        df_serv_ano_consolidado = pd.DataFrame()
        failed_siape_periods_ano = [] # Inicializa lista para rastrear períodos faltantes para este ano

        for mes_num in range(1, 13):
            mes_str = f'{mes_num:02d}'
            ano_mes = f'{ano}{mes_str}'
            url_zip_siape = f'https://dadosabertos-download.cgu.gov.br/PortalDaTransparencia/saida/servidores/{ano_mes}_Servidores_SIAPE.zip'
            nome_arquivo_zip_temp = os.path.join(temp_dir_ano, f'{ano_mes}_Servidores_SIAPE.zip')

            print(f"\n  -- Baixando arquivo SIAPE para {ano_mes} de: {url_zip_siape}")
            try:
                response = requests.get(url_zip_siape, stream=True)
                response.raise_for_status()

                with open(nome_arquivo_zip_temp, 'wb') as zip_file_temp:
                    for chunk in response.iter_content(chunk_size=8192):
                        zip_file_temp.write(chunk)
                print(f"  Arquivo ZIP baixado e salvo em: {nome_arquivo_zip_temp}")

                csv_path = extrair_csv_do_zip(nome_arquivo_zip_temp, pasta_temp=temp_dir_ano)
                if csv_path:
                    df_serv_mensal = carregar_dataframe_siape(csv_path, nome_arquivo_zip_temp)
                    if df_serv_mensal is not None:
                        df_serv_ano_consolidado = pd.concat([df_serv_ano_consolidado, df_serv_mensal], ignore_index=True)
                        try:
                            os.remove(csv_path)
                            print(f"  Arquivo CSV temporário removido: {csv_path}")
                        except Exception as e_remove_csv:
                            print(f"  Erro ao remover arquivo CSV temporário {csv_path}: {e_remove_csv}")
                    else:
                        print(f"  Erro ao carregar dados SIAPE do arquivo baixado: {nome_arquivo_zip_temp}, pulando.")
                else:
                    print(f"  Arquivo CSV não encontrado ou erro na extração do ZIP baixado: {nome_arquivo_zip_temp}, pulando.")

                try:
                    os.remove(nome_arquivo_zip_temp)
                    print(f"  Arquivo ZIP temporário removido: {nome_arquivo_zip_temp}")
                except Exception as e_remove_zip:
                    print(f"  Erro ao remover arquivo ZIP temporário {nome_arquivo_zip_temp}: {e_remove_zip}")

            except requests.exceptions.HTTPError as http_err:
                print(f"  Erro HTTP ao baixar {url_zip_siape}: {http_err}")
                failed_siape_periods_ano.append(ano_mes) # Adiciona ano_mes à lista de falhas
                continue
            except requests.exceptions.ConnectionError as conn_err:
                print(f"  Erro de conexão ao baixar {url_zip_siape}: {conn_err}")
                failed_siape_periods_ano.append(ano_mes) # Adiciona ano_mes à lista de falhas
                continue
            except requests.exceptions.Timeout as timeout_err:
                print(f"  Timeout ao baixar {url_zip_siape}: {timeout_err}")
                failed_siape_periods_ano.append(ano_mes) # Adiciona ano_mes à lista de falhas
                continue
            except Exception as e:
                print(f"  Erro inesperado ao baixar ou processar {url_zip_siape}: {e}")
                failed_siape_periods_ano.append(ano_mes) # Adiciona ano_mes à lista de falhas
                continue

        if not df_serv_ano_consolidado.empty:
            last_successful_df_serv_global = df_serv_ano_consolidado.copy()
            print(f"  DataFrame SIAPE consolidado para {ano} criado com {len(df_serv_ano_consolidado)} linhas.")
        else:
            print(f"  Erro ao consolidar dados SIAPE para {ano}. Usando último df_serv válido (se houver).")
            if last_successful_df_serv_global is not None:
                print(f"  Último df_serv válido disponível tem {len(last_successful_df_serv_global)} linhas.")
            else:
                print(f"  Nenhum df_serv válido disponível para usar como referência.")

        df_viagens_preparado_ano = preparar_dataframe_viagens(df_viagens)
        df_viagens_ano = df_viagens_preparado_ano[df_viagens_preparado_ano['Período - Data de início'].dt.year == int(ano)].copy()
        df_viagens_ano = df_viagens_ano.reset_index(drop=True)

        print(f"  Tamanho de df_viagens_ano antes do merge: {len(df_viagens_ano)}")
        print(f"  Tamanho de df_serv_ano antes do merge: {len(df_serv_ano_consolidado)}")

        df_merged_ano = merge_dataframes(df_viagens_ano, df_serv_ano_consolidado)

        if df_merged_ano is not None:
            print(f"  Tamanho de df_merged_ano após o merge: {len(df_merged_ano)}")
        else:
            print(f"  Erro no merge para o ano {ano}, df_merged_ano é None.")
            df_merged_ano = df_viagens_ano

        df_merged_ano.drop(columns=['NOME', 'siape_ano', 'siape_mes', 'viagem_ano_mes', 'viagem_ano', 'viagem_mes'], inplace=True, errors='ignore')
        print(f"Ano SIAPE {ano} processado (download e merge mensal sob demanda por NOME, ano e mês).")
        return df_merged_ano, failed_siape_periods_ano # Retorna também a lista de períodos faltantes para o ano
print('Função carregada com êxito')

Função carregada com êxito


In [83]:
# Célula 12: Função para Enriquecer Período Faltante - Modificada para receber lista de períodos faltantes
def enriquecer_periodo_faltante(df_viagens, last_successful_df_serv, failed_siape_periods):
    """Enriquece registros de viagem do período faltante com ORG_EXERCICIO da UFSC, usando o último df_serv válido.
       Agora recebe a lista de períodos faltantes como argumento.
    """
    if last_successful_df_serv is None:
        print("AVISO: Nenhum df_serv válido disponível para enriquecer o período faltante.")
        return df_viagens

    if not failed_siape_periods: # Verifica se a lista de períodos faltantes não está vazia
        print("Não foram identificados períodos faltantes de dados SIAPE.")
        return df_viagens # Retorna sem fazer nada se não houver períodos faltantes

    periodo_faltante_anos_meses = failed_siape_periods # Usa a lista de períodos faltantes passada como argumento
    df_viagens_periodo_faltante = df_viagens[df_viagens['Período - Data de início'].dt.strftime('%Y%m').isin(periodo_faltante_anos_meses)].copy()

    if df_viagens_periodo_faltante.empty:
        print("Não há registros de viagem para o período faltante identificado.")
        return df_viagens

    print(f"\nEnriquecendo {len(df_viagens_periodo_faltante)} registros de viagem do período faltante: {periodo_faltante_anos_meses}...")

    for index, row in df_viagens_periodo_faltante.iterrows():
        nome_viajante = row['Nome']
        servidor_ufsc = last_successful_df_serv[last_successful_df_serv['NOME'] == nome_viajante]
        if not servidor_ufsc.empty:
            df_viagens.loc[index, 'ORG_EXERCICIO'] = instituicao
            df_viagens.loc[index, 'FUNCAO'] = "Impreciso"
            df_viagens.loc[index, 'ATIVIDADE'] = "Impreciso"
            df_viagens.loc[index, 'UORG_EXERCICIO'] = "Impreciso"

    print(f"Enriquecimento do período faltante concluído.")
    return df_viagens
print('Função carregada com êxito')

Função carregada com êxito


In [84]:
# Célula 13: Inicialização - Download e Preparação dos DataFrames de Viagem (MODIFICADA)
# anos_para_baixar = ['2022', '2023', '2024'] # Anos para baixar os dados de viagem
dfs_viagens_filtrados = []

for ano in anos_para_baixar:
    print(f"\n--- Processando dados de viagem para o ano: {ano} ---")
    data_valida = encontrar_data_valida_viagens(ano) # Encontrar data válida primeiro

    if data_valida: # Se encontrou uma data válida
        zip_path = download_arquivo_viagens_com_data(ano, data_valida) # Baixar usando a data válida

        if zip_path:
            df_ano = carregar_dataframe_viagens_csv_zip(zip_path, ano)
            if df_ano is not None:
                dfs_viagens_filtrados.append(df_ano)

            try:
                os.remove(zip_path)
                print(f"Arquivo ZIP temporário removido: {zip_path}")
            except Exception as e_remove_zip:
                print(f"Erro ao remover arquivo ZIP temporário {zip_path}: {e_remove_zip}")
        else:
            print(f"Não foi possível baixar os dados de viagem para {ano} usando a data válida {data_valida}. Pulando para o próximo ano.")
    else:
        print(f"Não foi possível encontrar uma data válida para download dos dados de viagem para {ano}. Pulando para o próximo ano.")


--- Processando dados de viagem para o ano: 2022 ---
DEBUG: Iniciando busca por data válida para o ano: 2022. Data base inicial: 2025-03-09
DEBUG: Testando data 1/8: 20250309 - URL: https://dadosabertos-download.cgu.gov.br/PortalDaTransparencia/saida/viagens/2022_20250309_Viagens.zip
DEBUG: Acesso proibido (403) para https://dadosabertos-download.cgu.gov.br/PortalDaTransparencia/saida/viagens/2022_20250309_Viagens.zip na data 20250309. Tentando data anterior.
DEBUG: Testando data 2/8: 20250308 - URL: https://dadosabertos-download.cgu.gov.br/PortalDaTransparencia/saida/viagens/2022_20250308_Viagens.zip
DEBUG: Acesso proibido (403) para https://dadosabertos-download.cgu.gov.br/PortalDaTransparencia/saida/viagens/2022_20250308_Viagens.zip na data 20250308. Tentando data anterior.
DEBUG: Testando data 3/8: 20250307 - URL: https://dadosabertos-download.cgu.gov.br/PortalDaTransparencia/saida/viagens/2022_20250307_Viagens.zip
DEBUG: Acesso proibido (403) para https://dadosabertos-download.cg

In [85]:
# Célula 14: Inicialização - Concatenação dos DataFrames de Viagem e Consistência de Colunas
df_viagens_periodo = pd.concat(dfs_viagens_filtrados, ignore_index=True)

if dfs_viagens_filtrados:
    cols_2022 = dfs_viagens_filtrados[0].columns.tolist()
    for i in range(1, len(dfs_viagens_filtrados)):
        dfs_viagens_filtrados[i] = dfs_viagens_filtrados[i][cols_2022]
    df_viagens_periodo = pd.concat(dfs_viagens_filtrados, ignore_index=True)
    print("\nDataFrames de viagens concatenados e colunas consistentes garantidas.")
else:
    print("\nNão foi possível baixar e processar dados de viagem para nenhum ano. df_viagens_periodo não foi criado.")
    df_viagens_periodo = pd.DataFrame()


DataFrames de viagens concatenados e colunas consistentes garantidas.


In [86]:
# Célula 15: Preparar DataFrame de Viagens para Merge e Remover Colunas SIAPE Existentes
if not df_viagens_periodo.empty:
    df_viagens_periodo_preparado = preparar_dataframe_viagens(df_viagens_periodo)
    colunas_siape_para_remover = ['FUNCAO', 'ATIVIDADE', 'UORG_EXERCICIO', 'ORG_EXERCICIO']
    df_viagens_periodo_preparado = df_viagens_periodo_preparado.drop(columns=colunas_siape_para_remover, errors='ignore')
    print("\nDataFrame de viagens preparado para merge e colunas SIAPE removidas.")
else:
    print("\nDataFrame df_viagens_periodo está vazio. Pulando preparação para merge.")


DataFrame de viagens preparado para merge e colunas SIAPE removidas.


In [87]:
# Célula 16: Loop Principal - Processamento Anual SIAPE e Merge com Viagens
if not df_viagens_periodo.empty:
    df_viagens_periodo_atualizado = df_viagens_periodo_preparado.copy() # Inicia com o dataframe preparado
    all_failed_siape_periods = [] # Inicializa lista para acumular períodos faltantes de todos os anos

    for ano in anos_para_baixar:
        df_viagens_ano_processado, failed_periods_ano = processar_ano_refatorado(ano, df_viagens_periodo_atualizado, pasta_temp_raiz=pasta_temp_raiz_downloads) # Recebe df_merged e lista de falhas
        df_viagens_periodo_atualizado = pd.concat([df_viagens_periodo_atualizado[df_viagens_periodo_atualizado['Período - Data de início'].dt.year != int(ano)], df_viagens_ano_processado], ignore_index=True)
        all_failed_siape_periods.extend(failed_periods_ano) # Acumula períodos faltantes de cada ano

        if last_successful_df_serv_global is not None:
            last_successful_df_serv = last_successful_df_serv_global.copy()

    print("\nProcessamento anual SIAPE e merge com viagens concluído.")

    # Remover colunas 'viagem_ano_mes', 'viagem_ano', 'viagem_mes' ANTES do enriquecimento e de salvar
    colunas_para_remover_viagem = ['viagem_ano_mes', 'viagem_ano', 'viagem_mes']
    df_viagens_periodo_atualizado = df_viagens_periodo_atualizado.drop(columns=colunas_para_remover_viagem, errors='ignore')
    print("\nColunas 'viagem_ano_mes', 'viagem_ano', 'viagem_mes' removidas do DataFrame principal.")


else:
    print("\nDataFrame df_viagens_periodo está vazio. Pulando processamento SIAPE e merge.")


--- Processando ano SIAPE: 2022 ---
Pasta temporária para o ano SIAPE 2022: temp_downloads_raiz/tmp08ryhyla

  -- Baixando arquivo SIAPE para 202201 de: https://dadosabertos-download.cgu.gov.br/PortalDaTransparencia/saida/servidores/202201_Servidores_SIAPE.zip
  Arquivo ZIP baixado e salvo em: temp_downloads_raiz/tmp08ryhyla/202201_Servidores_SIAPE.zip
  Tentando ler CSV (com filtros): temp_downloads_raiz/tmp08ryhyla/202201_Cadastro.csv
  DataFrame SIAPE filtrado carregado com 959 linhas.
  Não foram encontradas duplicatas por NOME, siape_ano e siape_mes no arquivo SIAPE temp_downloads_raiz/tmp08ryhyla/202201_Cadastro.csv
  Arquivo CSV temporário removido: temp_downloads_raiz/tmp08ryhyla/202201_Cadastro.csv
  Arquivo ZIP temporário removido: temp_downloads_raiz/tmp08ryhyla/202201_Servidores_SIAPE.zip

  -- Baixando arquivo SIAPE para 202202 de: https://dadosabertos-download.cgu.gov.br/PortalDaTransparencia/saida/servidores/202202_Servidores_SIAPE.zip
  Arquivo ZIP baixado e salvo em: 

In [88]:
# Célula 17: Enriquecer Período Faltante - Modificada para receber lista de períodos faltantes
if not df_viagens_periodo_atualizado.empty:
    df_viagens_periodo_atualizado = enriquecer_periodo_faltante(df_viagens_periodo_atualizado, last_successful_df_serv, all_failed_siape_periods) # Passa lista de períodos faltantes
    print("\nEnriquecimento do período faltante concluído.")
else:
    print("\nDataFrame df_viagens_periodo_atualizado está vazio. Pulando enriquecimento do período faltante.")

Não foram identificados períodos faltantes de dados SIAPE.

Enriquecimento do período faltante concluído.


In [89]:
# Célula 18: Limpeza Final e Exibição dos Resultados
limpar_arquivos_temporarios() # Limpar pasta temp_siape_arquivos
try:
    shutil.rmtree(pasta_temp_raiz_downloads) # Remover pasta temp_downloads_raiz
    print(f"Pasta temporária raiz de downloads '{pasta_temp_raiz_downloads}' removida com sucesso.")
except FileNotFoundError:
    print(f"Pasta temporária raiz de downloads '{pasta_temp_raiz_downloads}' não encontrada, pulando remoção.")
except Exception as e:
    print(f"Erro ao remover pasta temporária raiz de downloads '{pasta_temp_raiz_downloads}': {e}")
try:
    shutil.rmtree(pasta_temp_viagens) # Remover pasta temp_viagens_download
    print(f"Pasta temporária de viagens '{pasta_temp_viagens}' removida com sucesso.")
except FileNotFoundError:
    print(f"Pasta temporária de viagens '{pasta_temp_viagens}' não encontrada, pulando remoção.")
except Exception as e:
    print(f"Erro ao remover pasta temporária de viagens '{pasta_temp_viagens}': {e}")


if not df_viagens_periodo_atualizado.empty:
    print("\nInformações do DataFrame df_viagens_periodo_atualizado:")
    df_viagens_periodo_atualizado.info()
    print("\nPrimeiras linhas de df_viagens_periodo_atualizado:")
    display(df_viagens_periodo_atualizado.head())
else:
    print("\nDataFrame df_viagens_periodo_atualizado está vazio.")

Pasta temporária 'temp_siape_arquivos' não encontrada, pulando remoção.
Pasta temporária raiz de downloads 'temp_downloads_raiz' removida com sucesso.
Pasta temporária de viagens 'temp_viagens_download' removida com sucesso.

Informações do DataFrame df_viagens_periodo_atualizado:


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55488 entries, 0 to 55487
Data columns (total 26 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   Identificador do processo de viagem  55488 non-null  int64         
 1   Número da Proposta (PCDP)            55488 non-null  object        
 2   Situação                             55488 non-null  object        
 3   Viagem Urgente                       55488 non-null  object        
 4   Justificativa Urgência Viagem        55484 non-null  object        
 5   Código do órgão superior             55488 non-null  int64         
 6   Nome do órgão superior               55488 non-null  object        
 7   Código órgão solicitante             55488 non-null  int64         
 8   Nome órgão solicitante               55488 non-null  object        
 9   CPF viajante                         55146 non-null  object        
 10  Nome      

,Identificador do processo de viagem,Número da Proposta (PCDP),Situação,Viagem Urgente,Justificativa Urgência Viagem,Código do órgão superior,Nome do órgão superior,Código órgão solicitante,Nome órgão solicitante,CPF viajante,...,Destinos,Motivo,Valor diárias,Valor passagens,Valor devolução,Valor outros gastos,FUNCAO,ATIVIDADE,UORG_EXERCICIO,ORG_EXERCICIO
0,17650202,000001/22,Realizada,NÃO,Sem informação,26000,Ministério da Educação,26258,Universidade Tecnológica Federal do Paraná,***.685.646-**,...,"Porto/Portugal, Bragança/Portugal, Viseu/Portugal",Esta missão internacional de trabalho está vin...,0.00,0.00,0.0,0.00,NaN,NaN,NaN,NaN
1,17748982,000101/22-1C,Realizada,NÃO,Sem informação,52000,Ministério da Defesa,52000,Ministério da Defesa - Unidades com vínculo di...,***.928.487-**,...,Montes Claros/MG,"Processo nº 60340.000481/2021-67, NOTA DE SERV...",0.00,2500.46,0.0,0.00,NaN,NaN,NaN,NaN
2,17752965,000182/22-1C,Realizada,NÃO,Sem informação,52000,Ministério da Defesa,52000,Ministério da Defesa - Unidades com vínculo di...,***.686.805-**,...,Montes Claros/MG,"Processo nº 60340.000481/2021-67, NOTA DE SERV...",0.00,3946.18,0.0,0.00,NaN,NaN,NaN,NaN
3,17763409,000002/22,Realizada,NÃO,Sem informação,26000,Ministério da Educação,26240,Universidade Federal da Paraíba,***.225.534-**,...,Brasília/DF,Participação de Reunião na EMBRAPA / CENARGEN ...,0.00,1334.31,0.0,0.00,NaN,NaN,NaN,NaN
4,17768015,000003/22-1C,Realizada,SIM,"A viagem foi cadastrada há tempo, somente tive...",26000,Ministério da Educação,26438,Instituto Federal de Santa Catarina,***.505.729-**,...,Lisboa/Portugal,Fortalecer a cooperação internacional entre o ...,31127.58,13742.83,0.0,2005.47,NaN,NaN,NaN,NaN


In [90]:
# Célula 19: Salvar DataFrame Filtrado para CSV com Nome Dinâmico

if not df_viagens_periodo_atualizado.empty:
    # 1. Filtrar linhas onde 'ORG_EXERCICIO' não é nulo
    df_viagens_periodo_atualizado_filtrado = df_viagens_periodo_atualizado.dropna(subset=['ORG_EXERCICIO']).copy()

    if not df_viagens_periodo_atualizado_filtrado.empty:
        # 2. Gerar nome do arquivo dinamicamente
        periodo_str = '_'.join(anos_para_baixar) # Junta os anos com underscore
        nome_arquivo_csv = f'df_viagens_periodo_atualizado_{periodo_str}.csv'

        # 3. Salvar para CSV (com parâmetros personalizados)
        df_viagens_periodo_atualizado_filtrado.to_csv(nome_arquivo_csv,
                                                    index=False,
                                                    sep=';',
                                                    encoding='iso-8859-1',
                                                    quotechar='"',
                                                    quoting=csv.QUOTE_ALL,
                                                    decimal=',')
        print(f"\nDataFrame filtrado (apenas com ORG_EXERCICIO preenchido) salvo para: {nome_arquivo_csv}")

    else:
        print("\nDataFrame filtrado (após enriquecimento) está vazio. Nenhum arquivo CSV foi gerado.")

else:
    print("\nDataFrame df_viagens_periodo_atualizado está vazio. Nenhum arquivo CSV foi gerado.")


DataFrame filtrado (apenas com ORG_EXERCICIO preenchido) salvo para: df_viagens_periodo_atualizado_2022_2023.csv
